In [38]:
n_user = 27094
n_item =42852

In [1]:
from collections import defaultdict
with open('/home/yamanishi/project/trip_recommend/LightGCN-PyTorch/data/jalan/train.txt','r') as f:
    all=f.read()
edge_from_train = []
edge_to_train = []
train_item_dict = defaultdict(list)
for line in all.split('\n'):
    line = line.split(' ')
    train_item_tmp = []
    for l in line[1:]:
        edge_from_train.append(int(line[0]))
        edge_to_train.append(int(l))
        train_item_tmp.append(int(l))
        train_item_dict[int(line[0])].append(int(l))

np.save('/home/yamanishi/project/trip_recommend/model/recommendation/data/train_edge.npy',np.array([edge_from_train, edge_to_train]),)

In [6]:
#making weight to each visit
from collections import defaultdict
with open('/home/yamanishi/project/trip_recommend/LightGCN-PyTorch/data/jalan/train.txt','r') as f:
    all=f.read()
weights = []
train_item_dict = defaultdict(list)
for line in all.split('\n'):
    line = line.split(' ')
    weight=np.linspace(0.5, 1, len(line))
    weights.append(weight)

weights = np.concatenate(weights)

In [8]:
np.save('/home/yamanishi/project/trip_recommend/model/popularity/data/train_weights.npy', weights)

In [3]:
import pickle
with open('./data/train_item.pkl', 'wb') as f:
    pickle.dump(train_item_dict, f)

In [5]:
from collections import defaultdict
with open('/home/yamanishi/project/trip_recommend/LightGCN-PyTorch/data/jalan/test.txt','r') as f:
    all=f.read()
edge_from_test = []
edge_to_test = []
test_item_dict = defaultdict(list)
for line in all.split('\n'):
    line = line.split(' ')
    test_item_tmp = []
    for l in line[1:]:
        edge_from_test.append(int(line[0]))
        edge_to_test.append(int(l))
        test_item_tmp.append(int(l))
        test_item_dict[int(line[0])].append(int(l))

np.save('/home/yamanishi/project/trip_recommend/model/recommendation/data/test_edge.npy',np.array([edge_from_test, edge_to_test]),)

In [7]:
import pickle
with open('./data/test_item.pkl', 'wb') as f:
    pickle.dump(test_item_dict, f)

In [8]:
test_item_dict

defaultdict(list,
            {0: [66, 341, 723, 2255],
             1: [23527, 23557, 23543, 24245, 30, 20, 0],
             2: [27383, 27373, 27439, 21072, 50, 21, 0],
             3: [50,
              21035,
              17348,
              17345,
              11429,
              10120,
              10039,
              31174,
              10002,
              17372,
              36286],
             4: [6636, 29538, 29537, 18163, 19, 0],
             5: [35227, 461, 35487, 35460, 19, 0],
             6: [9673, 6738, 6777, 6682, 6713, 6667],
             7: [0, 21, 370, 723],
             8: [2271, 2305, 2376, 2392],
             9: [2, 177, 58, 234, 149, 257, 0, 32, 33, 503, 28560],
             10: [63, 149, 0, 21, 144],
             11: [4557, 2274, 10230, 2257, 2267, 1320, 184, 10073],
             12: [14819, 14841, 18638, 18670],
             13: [1, 19, 46, 0, 32, 22],
             14: [6758, 6882, 6713, 6791, 32585],
             15: [4050, 3442, 3474, 4079, 4100, 42

In [43]:
user_x = torch.rand(n_user, 128)
torch.save(user_x, './data/user_x.pt')

In [44]:
with open('/home/yamanishi/project/trip_recommend/LightGCN-PyTorch/data/jalan/train.txt','r') as f:
    all=f.read()
edge_from_train = []
edge_to_train = []
for line in all.split('\n'):
    line = line.split(' ')
    for l in line[1:]:
        edge_from_train.append(int(line[0]))
        edge_to_train.append(int(l))

edge = np.array([edge_from_train, edge_to_train])

In [45]:
from tqdm import tqdm
from collections import defaultdict
user_pos, user_neg, item_pos, item_neg = [], [], [], []
neg_item_all = []
d_neg = defaultdict(list)
for i in tqdm(range(n_user)):
    pos_items = edge[1][edge[0]==i]
    #neg_items = np.random.choice(n_items, len(pos_items))
    all_items = np.arange(n_item)
    neg_items = all_items[~np.isin(all_items, pos_items)]
    neg_items = np.random.choice(all_items, len(pos_items)*1000, replace=True)
    neg_items = neg_items.reshape(1000, len(pos_items))
    item_pos.append(pos_items)
    neg_item_all.append(neg_items)
        #item_neg.append(neg_items)
neg_item_all=np.concatenate(neg_item_all, axis=1)
np.save('./data/neg_samples.npy', neg_item_all)

100%|██████████| 27094/27094 [00:42<00:00, 631.69it/s]


In [46]:
d_neg = {k:np.concatenate(v) for k,v in d_neg.items()}

In [47]:
for v in d_neg.values():
    print(len(v))

In [48]:
from tqdm import tqdm
from collections import defaultdict
user_pos, user_neg, item_pos, item_neg = [], [], [], []
neg_item_all = []
d_neg = defaultdict(list)
for i in tqdm(range(n_user)):
    pos_items = edge[1][edge[0]==i]
    user_pos+=[i]*len(pos_items)
    user_neg+=[i]*len(pos_items)
    #neg_items = np.random.choice(n_items, len(pos_items))
    all_items = np.arange(n_item)
    neg_items = all_items[~np.isin(all_items, pos_items)]
    neg_items = np.random.choice(all_items, len(pos_items), replace=False)
    item_pos.append(pos_items)
    item_neg.append(neg_items)
        #item_neg.append(neg_items)

100%|██████████| 27094/27094 [00:42<00:00, 632.59it/s]


In [49]:
user_pos = np.array(user_pos)
user_neg = np.array(user_neg)
item_pos = np.concatenate(item_pos)
item_neg = np.concatenate(item_neg)

In [50]:
torch.save(torch.from_numpy(user_pos).long(), '/home/yamanishi/project/trip_recommend/model/recommendation/data/user_pos.pt')
torch.save(torch.from_numpy(user_neg).long(), '/home/yamanishi/project/trip_recommend/model/recommendation/data/user_neg.pt')
torch.save(torch.from_numpy(item_pos).long(), '/home/yamanishi/project/trip_recommend/model/recommendation/data/item_pos.pt')
torch.save(torch.from_numpy(item_neg).long(), '/home/yamanishi/project/trip_recommend/model/recommendation/data/item_neg.pt')

In [51]:
torch.load('./data/item_pos.pt')

tensor([ 2905,  2914,  2922,  ..., 37161, 37428, 37085])

In [9]:
cell = torch.nn.GRUCell(128, 128)
print(cell)

GRUCell(128, 128)


In [11]:
cell.__dict__.keys()

dict_keys(['training', '_parameters', '_buffers', '_non_persistent_buffers_set', '_backward_hooks', '_is_full_backward_hook', '_forward_hooks', '_forward_pre_hooks', '_state_dict_hooks', '_load_state_dict_pre_hooks', '_modules', 'input_size', 'hidden_size', 'bias'])

In [14]:
cell._parameters.keys()

odict_keys(['weight_ih', 'weight_hh', 'bias_ih', 'bias_hh'])

In [15]:
cell._parameters['weight_ih'].size()

torch.Size([384, 128])

In [20]:
for v in cell._parameters.values():
    print(v.size())

torch.Size([384, 128])
torch.Size([384, 128])
torch.Size([384])
torch.Size([384])


In [22]:
cell(torch.rand(5,128), torch.rand(5, 128)).size()

torch.Size([5, 128])

In [28]:
import torch.nn as nn
class A(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a=10
        self.b=5
        self.c=7
        self.d = nn.Linear(10,2)

    def forward(self):
        self.a=9

In [33]:
a=A()
for k,v in a.state_dict().items():
    torch.nn.init.normal_(v, 0, 100)

In [39]:
class GRUCell(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.Wir = torch.nn.Parameter(torch.rand(input_size, hidden_size))
        self.bir = torch.nn.Parameter(torch.rand(hidden_size))
        self.Whr = torch.nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.bhr = torch.nn.Parameter(torch.rand(hidden_size))
        self.Wiz = torch.nn.Parameter(torch.rand(input_size, hidden_size))
        self.biz = torch.nn.Parameter(torch.rand(hidden_size))
        self.Whz = torch.nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.bhz = torch.nn.Parameter(torch.rand(hidden_size))
        self.Win = torch.nn.Parameter(torch.rand(input_size, hidden_size))
        self.bin = torch.nn.Parameter(torch.rand(hidden_size))
        self.Whn = torch.nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.bhn = torch.nn.Parameter(torch.rand(hidden_size))
        self.reset_parameters()

    def reset_parameters(self):
        for k,v in self.state_dict().items():
            torch.nn.init.normal_(v, 0, 100)
gru = GRUCell(40,40)
print(gru.state_dict())          

OrderedDict([('Wir', tensor([[ 116.1923,  -46.8045,   58.5816,  ...,   35.2601,  -32.4645,
           92.2274],
        [-180.2655,  -48.0828,  -33.5874,  ...,  -77.5870,  -44.9697,
          -31.6917],
        [  12.8908,   53.1832,   43.4094,  ...,    8.3584,  -48.5353,
           39.5062],
        ...,
        [  74.1303,  163.6485,   -1.1597,  ...,   71.7234,  -51.1708,
          221.0275],
        [-138.4073,  -65.1289,   34.4129,  ...,  182.4025,   39.1062,
          162.1084],
        [  49.8736,   85.7651,  200.5092,  ...,   29.7380,  -96.1720,
          -37.9107]])), ('bir', tensor([  69.7461,   58.1387,   58.1538, -132.0232, -120.0353, -158.5267,
         -86.3698,  -34.1405,   39.1620,  -28.5607,   14.7744, -249.7381,
         -72.2592,   22.6465,  -32.8449,   61.3561,   72.3175,   76.8818,
          21.0928,  -67.9639,   56.4471,  -15.8101,   21.1038,  -91.2462,
         166.3998,  -46.4860,    1.3340,   26.4630, -173.9817,  198.5480,
         -20.9383,    5.0457,  148.9157

In [40]:
s = torch.nn.Sigmoid()
s(torch.rand(3,4))

tensor([[0.6826, 0.6967, 0.6916, 0.7207],
        [0.6618, 0.5835, 0.6037, 0.6721],
        [0.5113, 0.7225, 0.6839, 0.7238]])

In [35]:
a.state_dict()

OrderedDict([('d.weight',
              tensor([[  12.9975,  -51.8216,  -63.2199,   82.0918,  245.8111, -158.5529,
                        -41.6100,  158.1048,   41.3909,  -12.4979],
                      [  19.4933,  -81.6028,   43.6149,  -27.6262, -143.3300,  -12.9748,
                         19.9827,   92.5556,  -67.2577, -170.7186]])),
             ('d.bias', tensor([155.5620,  48.5215]))])

In [41]:
torch.rand(3,2)@torch.rand(2,4)

tensor([[0.5830, 0.9416, 0.8240, 1.0253],
        [0.6361, 0.9614, 0.9064, 1.0526],
        [0.9494, 1.5594, 1.3389, 1.6957]])

In [42]:
a = torch.rand(2,2)
b = torch.rand(2,3)
print(a, b, a@b)

tensor([[0.3054, 0.1209],
        [0.9395, 0.8115]]) tensor([[0.2795, 0.3556, 0.9192],
        [0.5594, 0.1174, 0.7868]]) tensor([[0.1530, 0.1228, 0.3758],
        [0.7165, 0.4293, 1.5020]])


In [23]:
1-torch.rand(3,4)

tensor([[0.5345, 0.0715, 0.2813, 0.1706],
        [0.1312, 0.4250, 0.2433, 0.8246],
        [0.5673, 0.2407, 0.4252, 0.8107]])